##### Evaluer les pertes de terres naturelles et agricoles en 20 ans de croissance de l'agglomération nantaise
Traitement de la BDMOS de Loire-Atlantique, millésimes 1999, 2004, 2009, 2012, 2016 et 2020. 

*Source : https://data.loire-atlantique.fr/explore/dataset/224400028_modes-occupation-sols-loire-atlantique/information/?disjunctive.commune&disjunctive.epci&disjunctive.scot&disjunctive.insee_dep&disjunctive.delegation&disjunctive.ocs_1_1949&disjunctive.ocs_1_1999&disjunctive.ocs_2_1999&disjunctive.ocs_3_1999&disjunctive.ocs_1_2004&disjunctive.ocs_2_2004&disjunctive.ocs_3_2004&disjunctive.ocs_1_2009&disjunctive.ocs_2_2009&disjunctive.ocs_3_2009&disjunctive.ocs_2_2012&disjunctive.ocs_3_2012&disjunctive.ocs_1_2016&disjunctive.ocs_2_2016&disjunctive.ocs_3_2016&disjunctive.ocs_1_2020&disjunctive.ocs_2_2020&disjunctive.ocs_3_2020&sort=objectid*


In [9]:
import pandas as pd
import geopandas as gpd
from math import pi
import numpy as np
import fiona
import matplotlib as mpl
import matplotlib.pyplot as plt
pd.set_option("display.max_columns", None)

In [10]:
# Chargement du fichier geojson de la BDMOS : https://data.loire-atlantique.fr/explore/dataset/224400028_modes-occupation-sols-loire-atlantique/information/?disjunctive.commune&disjunctive.epci&disjunctive.scot&disjunctive.insee_dep&disjunctive.delegation&disjunctive.ocs_1_1949&disjunctive.ocs_1_1999&disjunctive.ocs_2_1999&disjunctive.ocs_3_1999&disjunctive.ocs_1_2004&disjunctive.ocs_2_2004&disjunctive.ocs_3_2004&disjunctive.ocs_1_2009&disjunctive.ocs_2_2009&disjunctive.ocs_3_2009&disjunctive.ocs_2_2012&disjunctive.ocs_3_2012&disjunctive.ocs_1_2016&disjunctive.ocs_2_2016&disjunctive.ocs_3_2016&disjunctive.ocs_1_2020&disjunctive.ocs_2_2020&disjunctive.ocs_3_2020&sort=objectid 
mos=gpd.read_file("MOS-loire-atlantique_1949_2020.geojson")

# Communes de l'aire d'attraction de Nantes :
aav=["44009","44018","44020","44024","44026","44035","44047","44074","44094","44101","44109","44114","44120","44143","44150","44162","44166","44171","44172","44190","44194","44198","44204","44215","44001","44002","44005","44014","44015","44016","44019","44023","44025","44027","44028","44029","44032","44033","44037","44039","44041","44043","44045","44048","44056","44061","44062","44063","44064","44066","44070","44071","44073","44076","44077","44079","44080","44082","44083","44084","44088","44089","44090","44091","44100","44102","44107","44108","44110","44111","44113","44115","44117","44119","44122","44127","44130","44133","44137","44138","44139","44140","44141","44142","44145","44149","44155","44156","44158","44159","44164","44165","44169","44173","44174","44178","44179","44186","44188","44195","44201","44202","44203","44205","44207","44208","44209","44212","44214","44216","44217","44220","44221","44223","44224"]
# Communes de Nantes Métropole :
nm=['44172', '44120', '44009', '44024', '44020', '44204', '44047','44166', '44114', '44094', '44109', '44171', '44194', '44150','44074', '44035', '44101', '44026', '44143', '44018', '44198','44162', '44215', '44190']
# Communes de l'aire d'attraction exterieures à Nantes Métropole :
hors_nm=["44001","44002","44005","44014","44015","44016","44019","44023","44025","44027","44028","44029","44032","44033","44037","44039","44041","44043","44045","44048","44056","44061","44062","44063","44064","44066","44070","44071","44073","44076","44077","44079","44080","44082","44083","44084","44088","44089","44090","44091","44100","44102","44107","44108","44110","44111","44113","44115","44117","44119","44122","44127","44130","44133","44137","44138","44139","44140","44141","44142","44145","44149","44155","44156","44158","44159","44164","44165","44169","44173","44174","44178","44179","44186","44188","44195","44201","44202","44203","44205","44207","44208","44209","44212","44214","44216","44217","44220","44221","44223","44224"]

# Ajout d'un variable qui précise le secteur gégraphique de chaque parcelle (Communes de Nantes Métropole / Autres communes de l'aire d'attraction)
mos=mos[mos['insee_com'].isin(aav)]
mos.loc[mos['insee_com'].astype('str').isin(nm), "SECTEUR"] = "NM"
mos.loc[mos['insee_com'].astype('str').isin(hors_nm), "SECTEUR"] = "HORS_NM"

In [11]:
# Ajout d'un variable indiquant la surface, en m2, de chaque parcelle :
mos=mos.to_crs('EPSG:2154')
mos['area']=mos['geometry'].area

# Transformation du code de niveau en chaîne de caractères : 
mos['ocs_code_3_1999']=mos['ocs_code_3_1999'].astype('str')
mos['ocs_code_3_2004']=mos['ocs_code_3_2004'].astype('str')
mos['ocs_code_3_2009']=mos['ocs_code_3_2009'].astype('str')
mos['ocs_code_3_2012']=mos['ocs_code_3_2012'].astype('str')
mos['ocs_code_3_2016']=mos['ocs_code_3_2016'].astype('str')
mos['ocs_code_3_2020']=mos['ocs_code_3_2020'].astype('str')

# Regroupements des modes d'occupations du sols en catégories :
artif=('111','112','113','114','115','121','122','123','124','131','132','133','141','142','143','144','152','153') 
# Dans les versions 1 et 2 de ce notebook, les catégories 143 et 144 (Chantier, terrains en mutation, Terrains vacants et friches urbaines) avaient été exclus : trop de faux positifs
habitat=('111','112','113','114','115')
transp=['131','132','133']
activ_commerc=('121','122')
services_urbains=('123')
agric=('211','221','222','223','224','231')
forets_landes_friches_broussailles=('311','321','322')
zones_humides=('411','412','413','414','415')
rivieres=('511','512')
# Attention, ici on va au-delà du périmètre habituel des "espaces naturels, agricoles et forestiers", en incluant les parcs et jardins urbains (catégorie 151) et les zones humides ('411','412','413','414','415')
vert=['151','211','221','222','223','224','231','311','321','322','411','412','413','414','415','511','512']

mos.sample(10)

,ocs_code_2_2009,ocs_2_2012,siren_epci,scot,ocs_code_3_2012,ocs_code_3_1999,ocs_1_2004,ocs_code_1_2012,ocs_1_2020,delegation,ocs_1_1999,ocs_code_1_2004,epci,insee_com,ocs_3_2009,ocs_3_2004,ocs_3_2020,ocs_2_2009,surface_ha,ocs_3_2016,ocs_code_1_1999,ocs_code_1_2009,ocs_code_3_2004,commune,insee_dep,ocs_2_2004,ocs_code_2_2012,ocs_1_2009,ocs_code_3_2016,ocs_code_1_2020,ocs_code_3_2009,ocs_1_2016,objectid,ocs_code_1_2016,ocs_3_1999,ocs_code_2_2016,ocs_code_2_1999,ocs_2_1999,ocs_2_2016,ocs_code_2_2004,ocs_code_1_1949,ocs_1_1949,ocs_2_2020,ocs_3_2012,ocs_code_3_2020,ocs_1_2012,ocs_code_2_2020,geometry,SECTEUR,area
10100,21,Prairies,244400537,SCOT du Pays de Châteaubriant,231,231,Territoires agricoles,2,Territoires agricoles,Délégation Châteaubriant,Territoires agricoles,2,CC de Nozay,44138,Cultures annuelles et terres labourées,Cultures annuelles et terres labourées,Prairies,Terres arables,4.478066,Prairies,2,2,211,Puceul,44,Terres arables,23,Territoires agricoles,231,2,211,Territoires agricoles,10530,2,Prairies,23,23,Prairies,Prairies,21,2,Territoires agricoles,Prairies,Prairies,231,Territoires agricoles,23,"POLYGON ((355433.859 6725027.737, 355364.408 6...",HORS_NM,44780.659751
369875,21,Prairies,200072734,SCOT de la métropole Nantes - Saint-Nazaire,231,211,Territoires agricoles,2,Territoires agricoles,Délégation Saint-Nazaire,Territoires agricoles,2,CC Estuaire et Sillon,44025,Cultures annuelles et terres labourées,Prairies,Prairies,Terres arables,4.006541,Prairies,2,2,231,Campbon,44,Prairies,23,Territoires agricoles,231,2,211,Territoires agricoles,397902,2,Cultures annuelles et terres labourées,23,21,Terres arables,Prairies,23,2,Territoires agricoles,Prairies,Prairies,231,Territoires agricoles,23,"POLYGON ((322334.256 6714600.310, 322514.540 6...",HORS_NM,40065.409911
380484,21,Terres arables,244400552,SCOT du Pays d'Ancenis,211,231,Territoires agricoles,2,Forêts et milieux semi naturels,Délégation Ancenis,Territoires agricoles,2,CC du Pays d'Ancenis,44077,Cultures annuelles et terres labourées,Cultures annuelles et terres labourées,Peuplements d'arbres en alignement,Terres arables,0.000009,Cultures annuelles et terres labourées,2,2,211,Joué-sur-Erdre,44,Terres arables,21,Territoires agricoles,211,3,211,Territoires agricoles,417795,2,Prairies,21,23,Prairies,Terres arables,21,2,Territoires agricoles,Bois et forêts,Cultures annuelles et terres labourées,312,Territoires agricoles,31,"POLYGON ((369616.953 6722384.551, 369537.719 6...",HORS_NM,0.094406
246996,11,Zones Urbanisées liées à l'habitat,200067866,SCOT du Pays du Vignoble nantais,112,112,Territoires artificialisés,1,Territoires artificialisés,Délégation vignoble,Territoires artificialisés,1,CC Sèvre et Loire,44108,Hameau rural,Hameau rural,Hameau rural,Zones Urbanisées liées à l'habitat,0.000005,Hameau rural,1,1,112,Mouzillon,44,Zones Urbanisées liées à l'habitat,11,Territoires artificialisés,112,1,112,Territoires artificialisés,267504,1,Hameau rural,11,11,Zones Urbanisées liées à l'habitat,Zones Urbanisées liées à l'habitat,11,2,Territoires agricoles,Zones Urbanisées liées à l'habitat,Hameau rural,112,Territoires artificialisés,11,"POLYGON ((377978.341 6676903.254, 377970.136 6...",HORS_NM,0.050482
283578,11,Zones Urbanisées liées à l'habitat,244400404,SCOT de la métropole Nantes - Saint-Nazaire,114,222,Territoires agricoles,1,Territoires artificialisés,Délégation Nantes,Territoires agricoles,2,Nantes Métropole,44198,Habitat pavillonnaire,"Maraîchage, horticulture",Habitat pavillonnaire,Zones Urbanisées liées à l'habitat,0.308112,Habitat pavillonnaire,2,1,222,Les Sorinières,44,Cultures permanentes et maraîchage,11,Territoires artificialisés,114,1,114,Territoires artificialisés,304086,1,"Maraîchage, horticulture",11,22,Cultures permanentes et maraîchage,Zones Urbanisées liées à l'habitat,22,2,Territoires agricoles,Zones Urbanisées liées à l'habitat,Habitat pavillonnaire,114,Territoires artificialisés,11,"POLYGON ((357754.613 6681606.781, 357751.447 6...",NM,3081.119196
14195

##### Artificialisation en Loire-Atlantique de 1999 à 2020

In [19]:
# Sélection, à Nantes Métropole, des espaces agricoles ou naturels en début de période devenus articiels :
mos_NM=mos[mos['SECTEUR']=='NM']
perte_vert_2020_NM=mos_NM[mos_NM['ocs_code_3_1999'].str.contains('|'.join(vert)) & mos_NM['ocs_code_3_2020'].str.contains('|'.join(artif))]
perte_vert_2020_NM=perte_vert_2020_NM[['ocs_code_3_1999','ocs_3_1999','ocs_code_3_2020','ocs_3_2020','area','insee_com','commune','SECTEUR','geometry']]
perte_vert_2020_NM=perte_vert_2020_NM[perte_vert_2020_NM['area']>=100]

# Export d'un tableur "origine destination" des espaces artificialisés" de Nantes Métropole:
bilan_croise_artif_NM_2020=perte_vert_2020_NM.pivot_table(values='area', index=['ocs_3_1999','ocs_3_2020'], aggfunc='sum')
bilan_croise_artif_NM_2020.to_csv('bilan_croise_artif_NM_1999-2020.csv', sep=";")

In [20]:
# Sélection des espaces agricoles ou naturels en début de période devenus articiels sur l'ensemble de l'aire d'attraction :
perte_vert_2020=mos[mos['ocs_code_3_1999'].str.contains('|'.join(vert)) & mos['ocs_code_3_2020'].str.contains('|'.join(artif))]
perte_vert_2020=perte_vert_2020[['ocs_code_3_1999','ocs_3_1999','ocs_code_3_2020','ocs_3_2020','area','insee_com','commune','SECTEUR','geometry']]
perte_vert_2020=perte_vert_2020[perte_vert_2020['area']>=100]

# Reprojection en WGS84 pour un usage web:
perte_vert_2020=perte_vert_2020.to_crs('EPSG:4326')
#perte_vert_2020.to_file('perte_vert_1999-2020.geojson')

# Calcul de la surface totale artificialisée : 
area=(perte_vert_2020['area'].sum())/1000000
area=area.round(decimals=2)
area=area.astype("str")
print("Entre 1999 et 2020, "+area+" km2 ont été artificialisés dans l'ensemble de l'aire d'attraction de Nantes")

# Export d'un tableur des pertes d'espaces agricoles ou naturels par commune et par type d'espaces :
bilan_perte_vert_2020=perte_vert_2020.pivot_table(values='area', index=['commune','ocs_3_1999'], aggfunc='sum')
#bilan_perte_vert_2020.to_csv('bilan_perte_vert_1999-2020.csv')
# Idem par secteurs (Métropole et reste de l'aire d'attraction) :
bilan_perte_vert_SECTEURS_2020=perte_vert_2020.pivot_table(values='area', index=['SECTEUR','ocs_3_1999'], aggfunc='sum')
#bilan_perte_vert_SECTEURS_2020.to_csv('SECTEURS_bilan_perte_vert_1999-2020.csv')

# Export d'un tableur des gains d'espaces artificiels par commune et par usage :
bilan_artif_2020=perte_vert_2020.pivot_table(values='area', index=['commune','ocs_3_2020'], aggfunc='sum')
#bilan_artif_2020.to_csv('bilan_artif_1999-2020.csv')
# Idem par secteurs (Métropole et reste de l'aire d'attraction) :
bilan_artif_SECTEURS_2020=perte_vert_2020.pivot_table(values='area', index=['SECTEUR','ocs_3_2020'], aggfunc='sum')
bilan_artif_SECTEURS_2020.to_csv('SECTEURS_bilan_artif_1999-2020.csv')

# Export d'un tableur "origine destination" des espaces artificialisés" :
bilan_croise_artif_2020=perte_vert_2020.pivot_table(values='area', index=['ocs_3_1999','ocs_3_2020'], aggfunc='sum')
##bilan_croise_artif_2020.to_csv('bilan_croise_artif_1999-2020.csv')

Entre 1999 et 2020, 87.12 km2 ont été artificialisés dans l'ensemble de l'aire d'attraction de Nantes


In [21]:
# Extraction des zones artificialisées pour chaque millésime :

artif_1999=mos[mos['ocs_code_3_1999'].str.contains('|'.join(artif))]
artif_1999=artif_1999
artif_1999=artif_1999.to_crs('EPSG:4326')
artif_1999=artif_1999[['commune','ocs_code_3_1999','ocs_3_1999','geometry','area','insee_com','SECTEUR']]
#artif_1999.to_file('BDMOS_44_1999_artif.geojson')

artif_2004=mos[mos['ocs_code_3_2004'].str.contains('|'.join(artif))]
artif_2004=artif_2004
artif_2004=artif_2004.to_crs('EPSG:4326')
artif_2004=artif_2004[['commune','ocs_code_3_2004','ocs_3_2004','geometry','area','insee_com','SECTEUR']]
#artif_2004.to_file('BDMOS_44_2004_artif.geojson')

artif_2009=mos[mos['ocs_code_3_2009'].str.contains('|'.join(artif))]
artif_2009=artif_2009
artif_2009=artif_2009.to_crs('EPSG:4326')
artif_2009=artif_2009[['commune','ocs_code_3_2009','ocs_3_2009','geometry','area','insee_com','SECTEUR']]
#artif_2009.to_file('BDMOS_44_2009_artif.geojson')

artif_2012=mos[mos['ocs_code_3_2012'].str.contains('|'.join(artif))]
artif_2012=artif_2012
artif_2012=artif_2012.to_crs('EPSG:4326')
artif_2012=artif_2012[['commune','ocs_code_3_2012','ocs_3_2012','geometry','area','insee_com','SECTEUR']]
#artif_2012.to_file('BDMOS_44_2012_artif.geojson')

artif_2016=mos[mos['ocs_code_3_2016'].str.contains('|'.join(artif))]
artif_2016=artif_2016
artif_2016=artif_2016.to_crs('EPSG:4326')
artif_2016=artif_2016[['commune','ocs_code_3_2016','ocs_3_2016','geometry','area','insee_com','SECTEUR']]
#artif_2016.to_file('BDMOS_44_2016_artif.geojson')

artif_2020=mos[mos['ocs_code_3_2020'].str.contains('|'.join(artif))]
artif_2020=artif_2020
artif_2020=artif_2020.to_crs('EPSG:4326')
artif_2020=artif_2020[['commune','ocs_code_3_2020','ocs_3_2020','geometry','area','insee_com','SECTEUR']]
#artif_2020.to_file('BDMOS_44_2020_artif.geojson')

In [22]:
# Extraction des zones agricoles ou naturelles en 1999 :

agric_1999=mos[mos['ocs_code_3_1999'].str.contains('|'.join(agric))]
agric_1999=agric_1999.to_crs('EPSG:4326')
#agric_1999.to_file('BDMOS_44_1999_agric.geojson')

forets_landes_friches_broussailles_1999=mos[mos['ocs_code_3_1999'].str.contains('|'.join(forets_landes_friches_broussailles))]
forets_landes_friches_broussailles_1999=forets_landes_friches_broussailles_1999.to_crs('EPSG:4326')
##forets_landes_friches_broussailles_1999.to_file('BDMOS_44_1999_forets_landes_friches_broussailles.geojson')

zones_humides_1999=mos[mos['ocs_code_3_1999'].str.contains('|'.join(zones_humides))]
zones_humides_1999=zones_humides_1999.to_crs('EPSG:4326')
zones_humides_1999.to_file('BDMOS_44_1999_zones_humides.geojson')

vert_1999=mos[mos['ocs_code_3_1999'].str.contains('|'.join(vert))]
vert_1999=vert_1999.to_crs('EPSG:4326')
#vert_1999.to_file('BDMOS_44_1999_vert.geojson')
vert_1999_plat=vert_1999.pivot_table(values='area', index=['commune','ocs_3_1999'], aggfunc='sum')
#vert_1999_plat.to_csv("BDMOS_44_1999_vert.csv")

# Extraction des zones déjà artificialisées en 1999 :
gris_1999=mos[mos['ocs_code_3_1999'].str.contains('|'.join(artif))]
gris_1999=gris_1999.to_crs('EPSG:4326')
#gris_1999.to_file('gris_1999.geojson')

##### Artificialisation entre 1999 et 2004

In [23]:
# Sélection des espaces agricoles ou naturels en début de période devenus articiels :
perte_vert_2004=mos[mos['ocs_code_3_1999'].str.contains('|'.join(vert)) & mos['ocs_code_3_2004'].str.contains('|'.join(artif))]
perte_vert_2004=perte_vert_2004[['ocs_code_3_1999','ocs_3_1999','ocs_code_3_2004','ocs_3_2004','area','insee_com','commune','SECTEUR','geometry']]
perte_vert_2004=perte_vert_2004[perte_vert_2004['area']>=100]

# Reprojection en WGS84 pour un usage web:
perte_vert_2004=perte_vert_2004.to_crs('EPSG:4326')
#perte_vert_2004.to_file('perte_vert_1999-2004.geojson')

# Calcul de la surface totale artificialisée : 
area=(perte_vert_2004['area'].sum())/1000000
area=area.round(decimals=2)
print('Entre 1999 et 2004, ', area,' km2 ont été artificialisés')
print(perte_vert_2004.shape)

# Export d'un tableur des pertes d'espaces agricoles ou naturels par commune et par type d'espaces :
bilan_perte_vert_2004=perte_vert_2004.pivot_table(values='area', index=['commune','ocs_3_1999'], aggfunc='sum')
#bilan_perte_vert_2004.to_csv('bilan_perte_vert_1999-2004.csv')

# Export d'un tableur des gains d'espaces artificiels par commune et par usage :
bilan_artif_2004=perte_vert_2004.pivot_table(values='area', index=['commune','ocs_3_2004'], aggfunc='sum')
#bilan_artif_2004.to_csv('bilan_artif_1999-2004.csv')

Entre 1999 et 2004,  29.18  km2 ont été artificialisés
(3849, 9)


##### Artificialisation entre 2004 et 2009

In [24]:
# Sélection des espaces agricoles ou naturels en début de période devenus artificiels :
perte_vert_2009=mos[mos['ocs_code_3_2004'].str.contains('|'.join(vert)) & mos['ocs_code_3_2009'].str.contains('|'.join(artif))]
perte_vert_2009=perte_vert_2009[['ocs_code_3_2004','ocs_3_2004','ocs_code_3_2009','ocs_3_2009','area','insee_com','commune','SECTEUR','geometry']]
perte_vert_2009=perte_vert_2009[perte_vert_2009['area']>=100]

# Reprojection en WGS84 pour un usage web:
perte_vert_2009=perte_vert_2009.to_crs('EPSG:4326')
#perte_vert_2009.to_file('perte_vert_2004-2009.geojson')

# Calcul de la surface totale artificialisée : 
area=(perte_vert_2009['area'].sum())/1000000
area=area.round(decimals=2)
print('Entre 2004 et 2009, ', area,' km2 ont été artificialisés')
print(perte_vert_2009.shape)

# Export d'un tableur des pertes d'espaces agricoles ou naturels par commune et par type d'espaces :
bilan_perte_vert_2009=perte_vert_2009.pivot_table(values='area', index=['commune','ocs_3_2004'], aggfunc='sum')
#bilan_perte_vert_2009.to_csv('bilan_perte_vert_2004-2009.csv')

# Export d'un tableur des gains d'espaces artificiels par commune et par usage :
bilan_artif_2009=perte_vert_2009.pivot_table(values='area', index=['commune','ocs_3_2009'], aggfunc='sum')
#bilan_artif_2009.to_csv('bilan_artif_2004-2009.csv')

Entre 2004 et 2009,  31.72  km2 ont été artificialisés
(6512, 9)


##### Artificialisation entre 2009 et 2012

In [25]:
# Sélection des espaces agricoles ou naturels en début de période devenus articiels :
perte_vert_2012=mos[mos['ocs_code_3_2009'].str.contains('|'.join(vert)) & mos['ocs_code_3_2012'].str.contains('|'.join(artif))]
perte_vert_2012=perte_vert_2012[['ocs_code_3_2009','ocs_3_2009','ocs_code_3_2012','ocs_3_2012','area','insee_com','commune','SECTEUR','geometry']]
perte_vert_2012=perte_vert_2012[perte_vert_2012['area']>=100]

# Reprojection en WGS84 pour un usage web:
perte_vert_2012=perte_vert_2012.to_crs('EPSG:4326')
#perte_vert_2012.to_file('perte_vert_2009-2012.geojson')

# Calcul de la surface totale artificialisée : 
area=(perte_vert_2012['area'].sum())/1000000
area=area.round(decimals=2)
print('Entre 2009 et 2012, ', area,' km2 ont été artificialisés')
print(perte_vert_2012.shape)

# Export d'un tableur des pertes d'espaces agricoles ou naturels par commune et par type d'espaces :
bilan_perte_vert_2012=perte_vert_2012.pivot_table(values='area', index=['commune','ocs_3_2009'], aggfunc='sum')
#bilan_perte_vert_2012.to_csv('bilan_perte_vert_2009-2012.csv')

# Export d'un tableur des gains d'espaces artificiels par commune et par usage :
bilan_artif_2012=perte_vert_2012.pivot_table(values='area', index=['commune','ocs_3_2012'], aggfunc='sum')
#bilan_artif_2012.to_csv('bilan_artif_2009-2012.csv')

Entre 2009 et 2012,  14.15  km2 ont été artificialisés
(3465, 9)


##### Artificialisation entre 2012 et 2016

In [26]:
# Sélection des espaces agricoles ou naturels en début de période devenus articiels :
perte_vert_2016=mos[mos['ocs_code_3_2012'].str.contains('|'.join(vert)) & mos['ocs_code_3_2016'].str.contains('|'.join(artif))]
perte_vert_2016=perte_vert_2016[['ocs_code_3_2012','ocs_3_2012','ocs_code_3_2016','ocs_3_2016','area','insee_com','commune','SECTEUR','geometry']]
perte_vert_2016=perte_vert_2016[perte_vert_2016['area']>=100]

# Reprojection en WGS84 pour un usage web:
perte_vert_2016=perte_vert_2016.to_crs('EPSG:4326')
#perte_vert_2016.to_file('perte_vert_2012-2016.geojson')

# Calcul de la surface totale artificialisée : 
area=(perte_vert_2016['area'].sum())/1000000
area=area.round(decimals=2)
print('Entre 2012 et 2016, ', area,' km2 ont été artificialisés')
print(perte_vert_2016.shape)

# Export d'un tableur des pertes d'espaces agricoles ou naturels par commune et par type d'espaces :
bilan_perte_vert_2016=perte_vert_2016.pivot_table(values='area', index=['commune','ocs_3_2012'], aggfunc='sum')
#bilan_perte_vert_2016.to_csv('bilan_perte_vert_2012-2016.csv')

# Export d'un tableur des gains d'espaces artificiels par commune et par usage :
bilan_artif_2016=perte_vert_2016.pivot_table(values='area', index=['commune','ocs_3_2016'], aggfunc='sum')
#bilan_artif_2016.to_csv('bilan_artif_2012-2016.csv')

Entre 2012 et 2016,  12.1  km2 ont été artificialisés
(2716, 9)


##### Artificialisation entre 2016 et 2020

In [27]:
# Sélection des espaces agricoles ou naturels en début de période devenus articiels :
perte_vert_2020=mos[mos['ocs_code_3_2016'].str.contains('|'.join(vert)) & mos['ocs_code_3_2020'].str.contains('|'.join(artif))]
perte_vert_2020=perte_vert_2020[['ocs_code_3_2016','ocs_3_2016','ocs_code_3_2020','ocs_3_2020','area','insee_com','commune','SECTEUR','geometry']]
perte_vert_2020=perte_vert_2020[perte_vert_2020['area']>=100]

# Reprojection en WGS84 pour un usage web:
perte_vert_2020=perte_vert_2020.to_crs('EPSG:4326')
#perte_vert_2020.to_file('perte_vert_2016-2020.geojson')

# Calcul de la surface totale artificialisée : 
area=(perte_vert_2020['area'].sum())/1000000
area=area.round(decimals=2)
print('Entre 2016 et 2020, ', area,' km2 ont été artificialisés')
print(perte_vert_2020.shape)

# Export d'un tableur des pertes d'espaces agricoles ou naturels par commune et par type d'espaces :
bilan_perte_vert_2020=perte_vert_2020.pivot_table(values='area', index=['commune','ocs_3_2016'], aggfunc='sum')
#bilan_perte_vert_2020.to_csv('bilan_perte_vert_2016-2020.csv')

# Export d'un tableur des gains d'espaces artificiels par commune et par usage :
bilan_artif_2020=perte_vert_2020.pivot_table(values='area', index=['commune','ocs_3_2020'], aggfunc='sum')
#bilan_artif_2020.to_csv('bilan_artif_2016-2020.csv')

Entre 2016 et 2020,  8.95  km2 ont été artificialisés
(1733, 9)


### Synthèse par secteurs (Nantes Métropole / reste de l'aire d'attraction)

In [28]:
###### 1999-2004 ##########
# Export d'un tableur des pertes d'espaces agricoles ou naturels par secteur et par type d'espaces :
bilan_perte_vert_SECTEURS_2004=perte_vert_2004.pivot_table(values='area', index=['SECTEUR','ocs_3_1999'], aggfunc='sum')
#bilan_perte_vert_SECTEURS_2004.to_csv('SECTEUR_bilan_perte_vert_1999-2004.csv')
# Export d'un tableur des gains d'espaces artificiels par secteur et par usage :
bilan_artif_SECTEURS_2004=perte_vert_2004.pivot_table(values='area', index=['SECTEUR','ocs_3_2004'], aggfunc='sum')
#bilan_artif_SECTEURS_2004.to_csv('SECTEUR_bilan_artif_1999-2004.csv')

###### 2004-2009 ##########
# Export d'un tableur des pertes d'espaces agricoles ou naturels par secteur et par type d'espaces :
bilan_perte_vert_SECTEURS_2009=perte_vert_2009.pivot_table(values='area', index=['SECTEUR','ocs_3_2004'], aggfunc='sum')
#bilan_perte_vert_SECTEURS_2009.to_csv('SECTEUR_bilan_perte_vert_2004-2009.csv')
# Export d'un tableur des gains d'espaces artificiels par secteur et par usage :
bilan_artif_SECTEURS_2009=perte_vert_2009.pivot_table(values='area', index=['SECTEUR','ocs_3_2009'], aggfunc='sum')
#bilan_artif_SECTEURS_2009.to_csv('SECTEUR_bilan_artif_2004-2009.csv')

###### 2009-2012 ##########
# Export d'un tableur des pertes d'espaces agricoles ou naturels par secteur et par type d'espaces :
bilan_perte_vert_SECTEURS_2012=perte_vert_2012.pivot_table(values='area', index=['SECTEUR','ocs_3_2009'], aggfunc='sum')
#bilan_perte_vert_SECTEURS_2012.to_csv('SECTEUR_bilan_perte_vert_2009-2012.csv')
# Export d'un tableur des gains d'espaces artificiels par secteur et par usage :
bilan_artif_SECTEURS_2012=perte_vert_2012.pivot_table(values='area', index=['SECTEUR','ocs_3_2012'], aggfunc='sum')
#bilan_artif_SECTEURS_2012.to_csv('SECTEUR_bilan_artif_2009-2012.csv')

###### 2012-2016 ##########
# Export d'un tableur des pertes d'espaces agricoles ou naturels par secteur et par type d'espaces :
bilan_perte_vert_SECTEURS_2016=perte_vert_2016.pivot_table(values='area', index=['SECTEUR','ocs_3_2012'], aggfunc='sum')
#bilan_perte_vert_SECTEURS_2016.to_csv('SECTEUR_bilan_perte_vert_2012-2016.csv')
# Export d'un tableur des gains d'espaces artificiels par secteur et par usage :
bilan_artif_SECTEURS_2016=perte_vert_2016.pivot_table(values='area', index=['SECTEUR','ocs_3_2016'], aggfunc='sum')
#bilan_artif_SECTEURS_2016.to_csv('SECTEUR_bilan_artif_2012-2016.csv')

###### 2016-2020 ##########
# Export d'un tableur des pertes d'espaces agricoles ou naturels par secteur et par type d'espaces :
bilan_perte_vert_SECTEURS_2020=perte_vert_2020.pivot_table(values='area', index=['SECTEUR','ocs_3_2016'], aggfunc='sum')
#bilan_perte_vert_SECTEURS_2020.to_csv('SECTEUR_bilan_perte_vert_2016-2020.csv')
# Export d'un tableur des gains d'espaces artificiels par secteur et par usage :
bilan_artif_SECTEURS_2020=perte_vert_2020.pivot_table(values='area', index=['SECTEUR','ocs_3_2020'], aggfunc='sum')
#bilan_artif_SECTEURS_2020.to_csv('SECTEUR_bilan_artif_2016-2020.csv')